<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: María Paz Raveau Morales</em><br>

</div>

# **<center>TAREA 2 COMPARANDO TOPIC MODELING Y CLUSTERING</center>**
*03 de agosto de 2025*

**Nombre Estudiante**: Cristian Tobar Morales  
**Asignatura**: Procesamiento de Lenguaje Natural


## **DESCRIPCIÓN DE LA PROBLEMÁTICA**
 

## **OBJETIVOS**

## **DESARROLLO**

### **CARGA Y ANÁLISIS EXPLORATORIO DE LOS DATOS**

#### Librerías requeridas

In [ ]:
import os
#%matplotlib inline
import pandas as pd
import numpy as np
import math


import string
from nltk.stem import SnowballStemmer
from wordcloud import WordCloud
from nltk import bigrams

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress

import re

# visualización 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import stanzas
tanza.download('es') # Ejecutar solamente una vez
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma,depparse')

In [ ]:
import nltk
nltk.download('punkt') # Ejecutar solamente una vez

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber','haber','hacer','poder','ir']
esp_stop.remove('estado')
esp_stop.remove('estados')

In [ ]:
import gensim
from gensim import corpora
from gensim import models

#### Cargar e Información inicial del dataset

In [ ]:
# Cargar los datos
url = 'ecqq.csv'

### **PRE-PROCESAMIENTO**

#### Tokenizar

#### Lemmatizar

#### Transformar el texto en minúsculas, eliminar puntuaciones, números, espacios en blanco y salto de líneas

#### Stopwords